# Root Finding for Systems of Equations: Gradient Descent
---

GENERAL PROBLEM: find the simultaneous roots of a system of $N$ non-linear functions 
$f_{1}(x_{1},\ldots,x_{N}), \dots, f_{N}(x_{1},\ldots,x_{N})$, in $N$ variables $x_{1},\ldots,x_{N}$. That is, find a combination of $x_{1},\ldots,x_{N}$ that satisfy the system of equations

\begin{align}
  \left.\begin{array}{l}
    f_{1}(x_{1}, \ldots, x_{N}) = 0 \\
    \quad\vdots \quad\quad\vdots \quad\quad\vdots \\
    f_{N}(x_{1}, \ldots, x_{N}) = 0
  \end{array}\right\}
  \quad\leftrightarrow\quad
  \mathbf{f}(\mathbf{x}) = \mathbf{0}
\end{align}

IDEA: Construct the function 
$g(\mathbf{x})\equiv \tfrac{1}{2}\mathbf{f}(\mathbf{x})\cdot\mathbf{f}(\mathbf{x})$, so that $\nabla{g}=\mathbf{J}(\mathbf{x})\cdot\mathbf{f}(\mathbf{x})$. Then locating a minimum of $g(\mathbf{x})$  (i.e., finding an $\mathbf{x}_{*}$ such that $\nabla{g}(\mathbf{x}_{*})=\mathbf{0}$) will correspond to finding a solution to the original problem, provided that $\mathbf{J}(\mathbf{x})$ is non-singular at $\mathbf{x}_{*}$.  

PRE-REQUISITES:   
- Newton-Raphson method for a single variable
- Solving systems of linear equations (LU decomposition, Gaussian elimination, etc)

REFERENCES:
- [1] Burden and Faires, *Numerical Analysis, 7th edition*.
- [2] Ralston and Rabinowitz, *A First Course in Numerical Analysis, 2nd edition*.
- [3] Press et al, *Numerical Recipes: the Art of Scientific Computing, 3rd edition*.
- [4] Stoer and Bulirsch, *Introduction to Numerical Analysis, 2nd edition*.

## 0. Review of Newton-Raphson for systems of equations

Recall how the Newton-Raphson method is applied to systems of non-linear equations. We want to find a solution to  

\begin{align}
  f_{1}(x_{1}, \ldots, x_{N}) = 0 \\
  \quad\vdots \quad\quad\vdots \quad\quad\vdots \\
  f_{N}(x_{1}, \ldots, x_{N}) = 0
\end{align}

which we can write in vector form as

\begin{align}
  \mathbf{f}(\mathbf{x}) = \mathbf{0}
  \quad,\quad\text{where}\quad\quad
  \mathbf{f}(\mathbf{x}) = 
    \left[\begin{array}{c}
      f_{1}(\mathbf{x})\\
      \vdots \\
      f_{N}(\mathbf{x})
    \end{array}\right]
  \quad,\quad
  \mathbf{x} = (x_{1}, \ldots, x_{N})
\end{align}

We expand $\mathbf{f}(\mathbf{x})$ in a Taylor series around an initial guess (approximant), truncate the series at linear order, and then rearrange to obtain an improved approximant. The result is the following two-step procedure that is carried out at each iteration 

\begin{align}
  &\text{Step 1:}\quad \mathbf{J}(\mathbf{x}_{i})\cdot\mathbf{\delta x}_{i} = -\mathbf{f}(\mathbf{x}_{i})
  \quad\text{(solve for $\mathbf{\delta x}_{i}$)}\\
  &\text{Step 2:}\quad \mathbf{x}_{i+1} = \mathbf{x}_{i} + \mathbf{\delta x}_{i}\
\end{align}

where $\mathbf{J}(\mathbf{x})$ is the Jacobian matrix given by

\begin{align}
  \mathbf{J}(\mathbf{x}) =
  \left[\begin{array}{ccc}
    \frac{\partial f_{1}}{\partial x_{1}}(\mathbf{x}) 
    & \cdots & \frac{\partial f_{1}}{\partial x_{N}}(\mathbf{x}) \\
    \vdots & \ddots & \vdots\\
    \frac{\partial f_{N}}{\partial x_{1}}(\mathbf{x}) 
    & \cdots & \frac{\partial f_{N}}{\partial x_{N}}(\mathbf{x}) \\
  \end{array}\right]
  \quad\quad\text{or}\quad\quad
  J_{ij}(\mathbf{x}) = \frac{\partial f_{i}}{\partial x_{j}}(\mathbf{x}).
\end{align}

Geometrically speaking, we are looking for the intersection point of the $N$ level-surfaces $f_{j}(x_{1},\ldots,x_{N})=0$ for $j=1,\ldots,N$, in the $N$-dimensional space $(x_{1},\ldots,x_{N})$. At each iteration Newton-Raphson has us take a step in this space, given by $\mathbf{\delta x}_{i}=\mathbf{J}^{-1}(\mathbf{x}_{i})\mathbf{f}(\mathbf{x}_{i})$. If we start with an initial guess that is sufficiently close to the solution, these steps take us closer and closer to the solution.

## 1. The gradient descent method

Recall that we want to solve the system of equations given by $\mathbf{f}(\mathbf{x}) = \mathbf{0}$. Define a new vector

\begin{align}
  g(\mathbf{x}) \equiv \tfrac{1}{2}\mathbf{f}(\mathbf{x})\cdot\mathbf{f}(\mathbf{x}).
\end{align}

The gradient is given by

\begin{align}
  \nabla{g} = \mathbf{f}(\mathbf{x})\cdot\mathbf{J}(\mathbf{x}),
\end{align}
  
where $\mathbf{J}(\mathbf{x})$ is the Jacobian matrix (defined above). Now notice that if $\mathbf{x}_{*}$ is a local minimum of $g(\mathbf{x})$, then $\nabla{g}(\mathbf{x}_{*})=\mathbf{0}$. If in addition the Jacobian is non-singular at $\mathbf{x}_{*}$, then $\mathbf{f}(\mathbf{x}_{*})=\mathbf{0}$. The point $\mathbf{x}_{*}$ is therefore a solution to our problem. So locating a minimum of $g(\mathbf{x})$ corresponds to finding a solution to the original problem, provided that the Jacobian is not singular there. The goal, then, is to minimize $g(\mathbf{x})$. This is done by stepping "downhill" in the direction of steepest descent (given by 
$-\nabla{g}(\mathbf{x_{i}})$) at each iteration until the minimum is reached.

What remains is to devise a scheme to determine *how large* of a step to take in the direction of steepest descent. In general the surface may drop and then rise again. We're guaranteed that a small enough step will take us downhill, but if we take too large a step, we may find ourselves having moved uphill rather than downhill. So start by defining $h(\alpha)$ as the value of $g(\mathbf{x})$ after taking a proposed step of size $\alpha$ in the 
$-\nabla{g}(\mathbf{x}_{i})$ direction, starting from $\mathbf{x}_{i}$:

\begin{align}
  h(\alpha) \equiv g\left(\mathbf{x}_{i} -\alpha\nabla{g}(\mathbf{x}_{i})\right)
  \quad,\quad \alpha > 0
\end{align}

Ideally we would like to take a step at each iteration so as to minimize $h(\alpha)$. That corresponds to moving as far downhill as possible from a given point, in the direction of steepest descent. However, finding this minimum at each iteration is computationally expensive. And so alternative strategies have been devised. Whatever scheme is used, the procedure is now summarized by the following two-step algorithm

\begin{align}
  &\text{Step 1:}\quad \text{(determine an appropriate $\alpha$)} \\
  &\text{Step 2:}\quad \mathbf{x}_{i+1} 
  = \mathbf{x}_{i} - \alpha\nabla{g}(\mathbf{x}_{i})
\end{align}

### Finding $\alpha$

There are numerous ways to determine $\alpha$. Here we will follow the scheme outlined by Burden and Faires [1]. The idea is to use three trial values for $\alpha$, perform a quadratic interpolation to a polynomial $P(\alpha)$ that approximates $h(\alpha)$, and then minimize $P(\alpha)$ which is much easier than minimizing $h(\alpha)$. More specifically, at each iteration we find three values of $\alpha$ that satisfy 
$h(\alpha)\leq g(\mathbf{x}_{i})$ such that 
$\alpha_{1} < \alpha_{2} < \alpha_{3}$. To reduce computational effort, take $\alpha_{1}=0$, since in that case $h(\alpha_{1})=g(\mathbf{x}_{i})$, and $g(\mathbf{x}_{i})$ is already calculated. Next set $\alpha_{3}=1$ and check whether it succeeds in decreasing $h(\alpha)$. If it does, accept it. If not, continue dividing by 2 until $h(\alpha_{3})< g(\mathbf{x}_{i})$. Recall that we are taking a step in the direction of 
$-\nabla{g}(\mathbf{x}_{i})$, so we are guaranteed to eventually find *some* value of $\alpha_{3}$ that is downhill. Once found, take $\alpha_{2}=\alpha_{3}/2$. With these values, we can calculate

\begin{align}
  & h_{1} \equiv h(\alpha_{1}) = g(\mathbf{x}_{i} - \alpha_{1}\nabla{g}(\mathbf{x}_{i})) \\
  & h_{2} \equiv h(\alpha_{2}) = g(\mathbf{x}_{i} - \alpha_{2}\nabla{g}(\mathbf{x}_{i})) \\
  & h_{3} \equiv h(\alpha_{3}) = g(\mathbf{x}_{i} - \alpha_{3}\nabla{g}(\mathbf{x}_{i}))
\end{align}

Next we use interpolation to find a quadratic curve through $h_{1}$, $h_{2}$, and $h_{3}$. Let this curve be denoted by $P(\alpha)$. Using Newton's forward divided-differences interpolating polynomial, this can be written generally as

\begin{align}
  P(\alpha) = c_{0} + c_{1}(\alpha - \alpha_{1}) + c_{2}(\alpha - \alpha_{1})(\alpha - \alpha_{2})
\end{align}

where

\begin{align}
  & c_{0} = h_{1} \\
  & c_{1} = \frac{(h_{2} - h_{1})}{(\alpha_{2} - \alpha_{1})} \\
  & c_{2} = \frac{(h_{3} - h_{1})/(\alpha_{3} - \alpha_{1}) - (h_{2} - h_{1})/(\alpha_{2} - \alpha_{1})}
            {\alpha_{3} - \alpha_{2}}
\end{align}

We have not imposed our choice of $\alpha_{1}=0$ in the expressions above. After we do, the expression simplifies slightly.

Finally, we minimize $P(\alpha)$ by taking a derivative, setting equal to zero, and solving for $\alpha$. This yields

\begin{align}
  \alpha = \frac{1}{2}\left(\alpha_{1} + \alpha_{2} - \frac{c_{1}}{c_{2}}\right)
\end{align}

This concludes all the formulae we need to carry out step 1 at each iteration.

PROGRAMMER'S NOTE: gradient descent is a "slow and steady" method. Its main advantage is that, unlike Newton-Raphson, it does not require a close initial guess in order to converge. However, it its main drawback is that it converges more slowly than Newton-Raphson. So just as in the single variable case, a prudent strategy is to couple a conservative method like gradient descent with a fast-and-loose method like Newton-Raphson. 

(See *Numerical Recipes* [3] for a scheme that incorporates gradient descent with a standard Newton-Raphson algorithm. At each step a decision is made whether to take a standard "Newton" step or whether to backtrack and take a smaller step using gradient descent.)

## 2. Algorithm

**INPUT**
- $\mathbf{x}_{0}$, initial guess for the solution of the system of equations.
- TOL, the relative error tolerance that the answer is required to have.
- $i_\mathrm{max}$, maximum number of iterations allowed.

**Initialize loop**
- set $i = 0$

**Loop** while $i \leq i_\mathrm{max}$


- calculate $\mathbf{f}(\mathbf{x}_{i})$, $\mathbf{J}(x_{i})$


- calculate $\nabla{g}(\mathbf{x}_{i}) = \mathbf{f}(\mathbf{x}_{i})\cdot\mathbf{J}(x_{i})$


- set $\alpha_{1}=0$, $h_{1}=g(\mathbf{x}_{i})$


- set $\alpha^{(0)}_{3}=1$, $k=0$, loop over $k$: 
    - calculate $h(\alpha^{(k)}_{3}) = g(\mathbf{x}_{i} - \alpha^{(k)}_{3}\nabla{g}(\mathbf{x}_{i}))$
    - test the downhill condition: $h(\alpha^{(k)}_{3})< g(\mathbf{x}_{i})$
    - continue until test is satisfied
    - set $\alpha_{3}=\alpha^{(k)}_{3}$, $h_{3}=h(\alpha^{(k)}_{3})$
  
  
- set $\alpha_{2}=\alpha_{3}/2$, $h_{2}=g(\mathbf{x}_{i}-\alpha_{2}\nabla{g}(\mathbf{x}_{i}))$


- calculate $c_{1}$ and $c_{2}$:
    - $c_{1} = \frac{(h_{2} - h_{1})}{(\alpha_{2} - \alpha_{1})}$
    - $c_{2} = \frac{(h_{3} - h_{1})/(\alpha_{3} - \alpha_{1}) - (h_{2} - h_{1})/(\alpha_{2} - \alpha_{1})}
            {\alpha_{3} - \alpha_{2}}$


- calculate $\alpha = \frac{1}{2}\left(\alpha_{1} + \alpha_{2} - \frac{c_{1}}{c_{2}}\right)$


- calculate $\mathbf{x}_{i+1} = \mathbf{x}_{i} - \alpha \nabla{g}(\mathbf{x}_{i})$


- save result


- calculate the relative uncertainty using: REL $= ||\mathbf{x}_{i+1} - \mathbf{x}_{i}||\,\,/\,\, ||\mathbf{x}_{i+1}||$


- calculate ABS $= ||\mathbf{f}(\mathbf{x}_{i+1})||$


- if (REL $\leq$ TOL) and (ABS $\leq$ TOL), stop. Otherwise, continue.


- rotate $\mathbf{x}_\mathrm{old} = \mathbf{x}_\mathrm{new}$


**Max iterations reached**
- Print message that max iterations have been reached, and stop.

**OUTPUT**

solution found, or message of failure

## 3. CODE:

In [13]:
%%writefile gradient_descent.py
import numpy as np
import numpy.linalg as la 
import sys
def gradient_descent(F, J, X0, TOL, imax):
    """
    Function that searches for solution of a system of equations,
    F(X)=0, using the gradient descent method.
    
    INPUT
    F    : function making up the system of equations 
    J    : Jacobian of first derivatives
    X0   : array of initial guesses for solution
    TOL  : allowed tolerance
    imax : maximum number of iterations
    
    OUTPUT
    solution to within the allowed tolerance, or failure message
    
    """
    
    # initialize output array
    solns = []
    
    # initialize iteration
    XOld = X0 # set initial approximant
    print('Initial guess for solution',X0)
    
    # iterate search using method of false position
    i = 0  # reset iteration number
    while i <= imax:
        
        # announce start of next iteration
        print('Iteration',i,':')
    
        # get function values
        FOld = F(XOld)

        # get Jacobian values
        JOld = J(XOld)
        detJ = abs(la.det(JOld)) 
        #print('  determinant of Jacobian is', detJ)
        if detJ < TOL:
            print('FAIL! Jacobian may be have hit a singularity. Stopping.')
            return
        
        # calculate g, grad(g)
        gOld = 0.5*np.dot(FOld, FOld)
        grad_gOld = np.dot(FOld, JOld) 
        
        # set alpha1, calculate h1
        alpha1 = 0
        X1 = XOld - alpha1*grad_gOld
        F1 = F(X1)
        h1 = 0.5*np.dot(F1, F1)
        #print('  alpha1 =',alpha1,'yields h1 =',h1)
        
        # find suitable alpha3, calculate h3
        alpha3 = 1.0
        X3 = XOld - alpha3*grad_gOld
        F3 = F(X3)
        h3 = 0.5*np.dot(F3, F3)
        #print('  alpha3 =',alpha3,'yields h3 =',h3)
        while h3 >= h1:
            #print('  downhill test failed, trying again...')
            alpha3 = 0.5*alpha3
            X3 = XOld - alpha3*grad_gOld
            F3 = F(X3)
            h3 = 0.5*np.dot(F3, F3)
            #print('  alpha3 =',alpha3,'yields h3 =',h3)
            if h3 == h1:
                print('FAIL! Could not find a downhill step within machine precision. Stopping.')
                return
        #print('  downhill test passed! moving on...')
        
        # get alpha2, calculate h2
        alpha2 = 0.5*alpha3
        X2 = XOld - alpha2*grad_gOld
        F2 = F(X2)
        h2 = 0.5*np.dot(F2, F2) 
        
        # calculate c1, c2
        c1 = (h2 - h1)/(alpha2 - alpha1)
        c2 = ((h3 - h1)/(alpha3 - alpha1) - (h2 - h1)/(alpha2 - alpha1))/(alpha3 - alpha2)
        
        # calculate alpha
        alpha = 0.5*(alpha1 + alpha2 - c1/c2)
        
        # update approximate location of root
        XNew = XOld - alpha*grad_gOld
        print('  approximate location of root at',XNew)
    
        # calculate errors
        XErr = la.norm(XNew - XOld, 2)
        REL  = XErr/la.norm(XNew, 2)
        ABS  = la.norm(F(XNew), 2)

        # save approximant and error
        solns.append([XNew, REL, ABS])

        # check if errors are within the allowed tolerance
        if (REL <= TOL and ABS <= TOL):
            best   = XNew  #best estimate
            uncert = XErr  #uncertainty
            print('SUCCESS! Solution found within the specified tolerance after',i,'iterations.')
            print('Solution is',best,'+/-',uncert)
            return solns
                
        # rotate approximants
        XOld = XNew

        # increment iteration number
        i = i + 1
        
    # print message that max iteration has been reached
    print('FAIL! Max number of iterations has been reached. Stopping.')
    
    return solns

Overwriting gradient_descent.py


In [14]:
%run gradient_descent.py

## 4. A simple 2D example

(Ralston and Rabinowitz, example 8.7)

The system of equations

\begin{align}
  & f_{1}(x, y) = x^2 - y - 1 \\
  & f_{2}(x, y) = (x - 2)^2 + (y - 0.5)^2 - 1
\end{align}

has two solutions

\begin{align}
  & r_{1} = [1.54634288332, 1.39117631279] \\
  & r_{2} = [1.06734608581, 0.139227666887]
\end{align}

In [15]:
# system of equations
def system(XX):
    x = XX[0]
    y = XX[1]
    dim = len(XX)
    f = np.zeros(dim)
    f[0] = x*x - y - 1 
    f[1] = (x - 2)**2 + (y - 0.5)**2 - 1
    return f

# Jacobian matrix elements
def jacobian(XX):
    x = XX[0]
    y = XX[1]
    dim = len(XX)
    dfdx = np.zeros((dim, dim))
    dfdx[0,0] = 2*x
    dfdx[0,1] = -1
    dfdx[1,0] = 2*(x - 2)
    dfdx[1,1] = 2*(y - 0.5)
    return dfdx

In [18]:
# exact solutions
exact1 = np.array([1.54634288332, 1.39117631279])
exact2 = np.array([1.06734608581, 0.139227666887])

# problematic point
[1.224, 0.695]

# solve system
#X0 = np.array([1.0, 2.0])
X0 = np.array([100.0, 100.0])
TOL = 1e-3
IMAX = 100
solns = gradient_descent(system, jacobian, X0, TOL, IMAX)

Initial guess for solution [100. 100.]
Iteration 0 :
  approximate location of root at [-20.59351044  19.54288242]
Iteration 1 :
  approximate location of root at [0.5518816  7.15988919]
Iteration 2 :
  approximate location of root at [2.12895453 0.26684498]
Iteration 3 :
  approximate location of root at [1.07585136 0.48512503]
Iteration 4 :
  approximate location of root at [1.13985987 0.43635012]
Iteration 5 :
  approximate location of root at [1.05546944 0.32456973]
Iteration 6 :
  approximate location of root at [1.10221682 0.28740276]
Iteration 7 :
  approximate location of root at [1.05647293 0.22951049]
Iteration 8 :
  approximate location of root at [1.08329864 0.20855856]
Iteration 9 :
  approximate location of root at [1.06141618 0.18042317]
Iteration 10 :
  approximate location of root at [1.07445156 0.17020547]
Iteration 11 :
  approximate location of root at [1.06453433 0.15752632]
Iteration 12 :
  approximate location of root at [1.07047351 0.15284929]
Iteration 13 :
  a

In [20]:
# print solutions
iterations = len(solns)
print('x \t\t y \t\t rel_error \t abs_error')
for i in range(iterations):
    xsoln = solns[i][0][0]
    ysoln = solns[i][0][1]
    rel_err = solns[i][1]
    abs_err = solns[i][2]
    print('%.8f \t %.8f \t %.8f \t %.8f' % (xsoln, ysoln, rel_err, abs_err))

x 		 y 		 rel_error 	 abs_error
-20.59351044 	 19.54288242 	 5.10627761 	 960.94094656
0.55188160 	 7.15988919 	 3.41233486 	 46.12499245
2.12895453 	 0.26684498 	 3.29563395 	 3.39517566
1.07585136 	 0.48512503 	 0.91129805 	 0.35861338
1.13985987 	 0.43635012 	 0.06593400 	 0.29048098
1.05546944 	 0.32456973 	 0.12683691 	 0.22422148
1.10221682 	 0.28740276 	 0.05243034 	 0.16552061
1.05647293 	 0.22951049 	 0.06824767 	 0.11913425
1.08329864 	 0.20855856 	 0.03085432 	 0.08252113
1.06141618 	 0.18042317 	 0.03310598 	 0.05641924
1.07445156 	 0.17020547 	 0.01522516 	 0.03801599
1.06453433 	 0.15752632 	 0.01495825 	 0.02545876
1.07047351 	 0.15284929 	 0.00699106 	 0.01695079
1.06607842 	 0.14726262 	 0.00660498 	 0.01125481
1.06871431 	 0.14518134 	 0.00311398 	 0.00745472
1.06678604 	 0.14273814 	 0.00289184 	 0.00493164
1.06794295 	 0.14182349 	 0.00136896 	 0.00325911
1.06710077 	 0.14075804 	 0.00126178 	 0.00215265
1.06760612 	 0.14035828 	 0.00059840 	 0.00142119
1.06723902 	

## 5. A simple 3D example

(Burden and Faires, example 10.2.1)

The system of equations

\begin{align}
  & f_{1}(x, y, z) = 3x - \cos(yz) - 0.5 \\
  & f_{2}(x, y, z) = x^2 - 81(y + 0.1)^2 + \sin(z) + 1.06\\
  & f_{3}(x, y, z) = \exp(-xy) + 20z + \frac{10\pi - 3}{3}
\end{align}

has a solution near

\begin{align}
  r_{1} = [0.5, 0.0, -0.52359877]
\end{align}

In [21]:
# system of equations
def system(XX):
    x = XX[0]
    y = XX[1]
    z = XX[2]
    dim = len(XX)
    f = np.zeros(dim)
    f[0] = 3*x - np.cos(y*z) - 0.5 
    f[1] = x**2 - 81.0*(y + 0.1)**2 + np.sin(z) + 1.06
    f[2] = np.exp(-x*y) + 20.0*z + (10.0*np.pi - 3.0)/3.0
    return f

# Jacobian matrix elements
def jacobian(XX):
    x = XX[0]
    y = XX[1]
    z = XX[2]
    dim = len(XX)
    dfdx = np.zeros((dim, dim))
    dfdx[0,0] = 3.0
    dfdx[0,1] = z*np.sin(y*z)
    dfdx[0,2] = y*np.sin(y*z)
    
    dfdx[1,0] = 2.0*x
    dfdx[1,1] = -162.0*(y + 0.1)
    dfdx[1,2] = np.cos(z)

    dfdx[2,0] = -y*np.exp(x*y)
    dfdx[2,1] = -x*np.exp(-x*y)
    dfdx[2,2] = 20.0

    return dfdx

In [32]:
# exact solutions
exact1 = np.array([0.5, 0.0, -0.52359877])

# solve system
#X0 = np.array([0.1, 0.1, -0.1])
X0 = np.array([0.0, 0.0, 0.0])
TOL = 1e-3
IMAX = 200
solns = gradient_descent(system, jacobian, X0, TOL, IMAX)

Initial guess for solution [0. 0. 0.]
Iteration 0 :
  approximate location of root at [ 0.01121615  0.01009453 -0.52264641]
Iteration 1 :
  approximate location of root at [ 0.45783804 -0.74984803 -0.52407546]
Iteration 2 :
  approximate location of root at [ 0.4634405  -0.10437905 -0.52023721]
Iteration 3 :
  approximate location of root at [ 0.46236706 -0.10569141 -0.52818774]
Iteration 4 :
  approximate location of root at [ 0.4010933  -0.2263289  -0.50002098]
Iteration 5 :
  approximate location of root at [ 0.4043288  -0.1951826  -0.52938584]
Iteration 6 :
  approximate location of root at [ 0.40938194 -0.19397278 -0.52540605]
Iteration 7 :
  approximate location of root at [ 0.41270351 -0.19463871 -0.52941364]
Iteration 8 :
  approximate location of root at [ 0.4171481  -0.19478676 -0.52570043]
Iteration 9 :
  approximate location of root at [ 0.42020553 -0.19505087 -0.52936368]
Iteration 10 :
  approximate location of root at [ 0.42424511 -0.19518226 -0.52597799]
Iteration 11 :


  approximate location of root at [ 0.49762494 -0.19957313 -0.52880832]
Iteration 121 :
  approximate location of root at [ 0.49764888 -0.19957515 -0.52883147]
Iteration 122 :
  approximate location of root at [ 0.49766964 -0.19957595 -0.52880988]
Iteration 123 :
  approximate location of root at [ 0.49769161 -0.1995778  -0.52883104]
Iteration 124 :
  approximate location of root at [ 0.49771052 -0.19957853 -0.5288113 ]
Iteration 125 :
  approximate location of root at [ 0.49773067 -0.19958023 -0.52883064]
Iteration 126 :
  approximate location of root at [ 0.49774789 -0.19958088 -0.5288126 ]
Iteration 127 :
  approximate location of root at [ 0.49776639 -0.19958244 -0.52883027]
Iteration 128 :
  approximate location of root at [ 0.49778207 -0.19958304 -0.52881378]
Iteration 129 :
  approximate location of root at [ 0.49779904 -0.19958446 -0.52882994]
Iteration 130 :
  approximate location of root at [ 0.49781332 -0.19958501 -0.52881486]
Iteration 131 :
  approximate location of root a

In [33]:
# print results
iterations = len(solns)
print('x \t\t y \t\t z \t\t rel_error \t abs_error')
for i in range(iterations):
    xsoln = solns[i][0][0]
    ysoln = solns[i][0][1]
    zsoln = solns[i][0][2]
    rel_err = solns[i][1]
    abs_err = solns[i][2]
    print('%.8f \t %.8f \t %.8f \t %.8f \t %.8f' % (xsoln, ysoln, zsoln, rel_err, abs_err))

x 		 y 		 z 		 rel_error 	 abs_error
0.01121615 	 0.01009453 	 -0.52264641 	 1.00000000 	 1.52564945
0.45783804 	 -0.74984803 	 -0.52407546 	 0.86164437 	 33.43972771
0.46344050 	 -0.10437905 	 -0.52023721 	 0.91626065 	 0.79229984
0.46236706 	 -0.10569141 	 -0.52818774 	 0.01145158 	 0.77634849
0.40109330 	 -0.22632890 	 -0.50002098 	 0.20330716 	 0.84212607
0.40432880 	 -0.19518260 	 -0.52938584 	 0.06184433 	 0.28409559
0.40938194 	 -0.19397278 	 -0.52540605 	 0.00943437 	 0.27090201
0.41270351 	 -0.19463871 	 -0.52941364 	 0.00750814 	 0.25865473
0.41714810 	 -0.19478676 	 -0.52570043 	 0.00829065 	 0.24706099
0.42020553 	 -0.19505087 	 -0.52936368 	 0.00679338 	 0.23595774
0.42424511 	 -0.19518226 	 -0.52597799 	 0.00749590 	 0.22539255
0.42704229 	 -0.19542636 	 -0.52931847 	 0.00616679 	 0.21527314
0.43071363 	 -0.19554898 	 -0.52623090 	 0.00678172 	 0.20564417
0.43327321 	 -0.19577280 	 -0.52927737 	 0.00560150 	 0.19642044
0.43661007 	 -0.19588687 	 -0.52646137 	 0.00613927 	